In [7]:
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf

#les données de prix de pétrole et des devises
oil_data = yf.download('CL=F', start='2015-01-01', end='2024-01-01')
nok_data = yf.download('NOK=X', start='2015-01-01', end='2024-01-01')
rub_data = yf.download('RUB=X', start='2015-01-01', end='2024-01-01')

data = pd.DataFrame({
    'Oil Price': oil_data['Close'],
    'NOK': nok_data['Close'],
    'RUB': rub_data['Close']
}).dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Graphique du prix du pétrole et des petro-monnaies
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=data.index, y=data['Oil Price'], mode='lines', name='Oil Price'))
fig1.add_trace(go.Scatter(x=data.index, y=data['NOK'], mode='lines', name='NOK'))
fig1.add_trace(go.Scatter(x=data.index, y=data['RUB'], mode='lines', name='RUB'))
fig1.update_layout(title="Oil Price vs NOK and RUB", xaxis_title="Date", yaxis_title="Price")
fig1.show()

Le graph permet de visualiser comment les deux devises de pays exportateurs de pétrole réagissent aux fluctuations du prix du pétrole. Les variations du RUB semblent être plus en phase avec celles du pétrole par rapport au NOK, ce qui pourrait être lié aux différences structurelles et économiques entre la Russie et la Norvège.

La période de 2022 sur ce graphique montre une volatilité marquée dans les prix du pétrole (en bleu) et une réponse sensible des devises associées au pétrole, comme le rouble russe (RUB en vert). La guerre en Ukraine, déclenchée en février 2022, a provoqué une série de sanctions économiques contre la Russie de la part des pays occidentaux. Ces sanctions ont eu un impact immédiat sur le rouble, entraînant d'importantes fluctuations.

Face aux sanctions, la Russie a cherché à stabiliser sa devise en liant partiellement ses transactions énergétiques, notamment le gaz et le pétrole, au rouble. Cela a contribué à une remontée du rouble après sa chute initiale. Le prix du pétrole, quant à lui, a subi une forte hausse à la suite des préoccupations concernant l'approvisionnement mondial en énergie, la Russie étant l'un des plus grands exportateurs de pétrole et de gaz. Cette dynamique a créé une instabilité importante sur le marché, reflétée par des pics et des baisses soudains dans le prix du pétrole et la volatilité des devises pétrolières.

In [ ]:
# Graphique de corrélation entre le pétrole et NOK
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=data['Oil Price'], y=data['NOK'], mode='markers', name='Oil vs NOK'))
fig2.update_layout(title="Correlation between Oil Price and NOK", xaxis_title="Oil Price", yaxis_title="NOK Price")
fig2.show()

On observe ici une tendance générale qui pourrait indiquer une corrélation entre les deux variables, mais la dispersion importante des points suggère également la présence d’autres facteurs influençant le NOK, tels que des événements géopolitiques ou des variables macroéconomiques.

In [ ]:
#  Volatilité du pétrole
data['Oil Volatility'] = data['Oil Price'].pct_change().rolling(window=30).std()
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=data.index, y=data['Oil Volatility'], mode='lines', name='Oil Volatility'))
fig3.update_layout(title="Oil Price Volatility (30-day Rolling)", xaxis_title="Date", yaxis_title="Volatility")
fig3.show()

In [ ]:
#  Volatilité du NOK
data['NOK Volatility'] = data['NOK'].pct_change().rolling(window=30).std()
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=data.index, y=data['NOK Volatility'], mode='lines', name='NOK Volatility'))
fig4.update_layout(title="NOK Volatility (30-day Rolling)", xaxis_title="Date", yaxis_title="Volatility")
fig4.show()

Des pics dans la volatilité indiquent des périodes d'incertitude accrue, souvent associées à des événements géopolitiques ou économiques significatifs. Par exemple sur ce graph, ainsi que sur le graph de la volatilité on constate les conséquences directes de la crise du COVID-19. Celle ci a brutalement réduit la demande mondiale de pétrole, en particulier dans les secteurs des transports et de l'industrie, suite aux confinements et aux restrictions de déplacement. Ce choc de demande a été aggravé par une guerre des prix entre la Russie et l'Arabie Saoudite en mars, lorsque les deux pays n'ont pas réussi à s'entendre sur des réductions de production via l'OPEP+. En réaction, l'Arabie Saoudite a augmenté sa production, provoquant une surabondance d'offre sur un marché déjà en déclin, ce qui a précipité la chute des prix.

La situation a culminé en avril 2020 lorsque les prix des contrats à terme sur le pétrole WTI (West Texas Intermediate) ont brièvement plongé en territoire négatif pour la première fois de l'histoire. Cela s'explique par une saturation des capacités de stockage, les investisseurs ne trouvant plus de place pour entreposer le pétrole acheté. Les coûts de stockage ont explosé, et les opérateurs de marché se sont retrouvés à payer pour se débarrasser de leurs contrats à terme afin d'éviter des livraisons physiques impossibles à gérer.

Cette crise a mis en lumière la vulnérabilité des marchés pétroliers aux chocs externes, la rigidité des infrastructures de stockage, et l'impact des décisions géopolitiques sur la stabilité des prix du pétrole.

In [ ]:
#  Modèle de régression et résidus
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = data[['Oil Price']]
y = data['NOK']

# Diviser en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
model = ElasticNet(alpha=0.1, l1_ratio=0.5).fit(X_train, y_train)
y_pred = model.predict(X_test)

fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=y_test.index, y=y_test, mode='lines', name='True NOK'))
fig5.add_trace(go.Scatter(x=y_test.index, y=y_pred, mode='lines', name='Predicted NOK'))
fig5.update_layout(title="Elastic Net Regression on NOK with Oil Price", xaxis_title="Date", yaxis_title="NOK Price")
fig5.show()

# Calcul des résidus
residuals = y_test - y_pred
fig6 = go.Figure()
fig6.add_trace(go.Scatter(x=y_test.index, y=residuals, mode='lines', name='Residuals'))
fig6.update_layout(title="Residuals of Elastic Net Model", xaxis_title="Date", yaxis_title="Residuals")
fig6.show()

le modèle parvient à capturer les tendances générales du NOK, bien que certaines variations brusques ne soient pas parfaitement modélisées, indiquant une limitation du modèle dans la capture de tous les facteurs influents.

Les résidus du modèle représentent la différence entre les valeurs réelles et prédites du NOK par le modèle. Ce graphique montre que les résidus oscillent relativement autour de [1,2], ce qui est attendu pour un modèle bien ajusté. Cependant, des pics dans les résidus indiquent des moments où le modèle ne parvient pas à capturer certaines variations, probablement dues à des événements exogènes ou des changements dans les conditions du marché.